In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
from plot_functions import *

# API base URL
API_BASE_URL = "https://localhost:9090/api"

# Function to get suburbs
def get_suburbs():
    response = requests.get(f"{API_BASE_URL}/suburbs")
    return response.json() if response.status_code == 200 else []

# Function to get crime data for a suburb
def get_crime_data(suburb):
    response = requests.get(f"{API_BASE_URL}/crime/{suburb}")
    return response.json() if response.status_code == 200 else {}

# Function to get weather data for a suburb
def get_weather_data(suburb):
    response = requests.get(f"{API_BASE_URL}/weather/{suburb}")
    return response.json() if response.status_code == 200 else {}

# Function to get crime and weather data for a suburb
def get_crime_weather_data(suburb):
    response = requests.get(f"{API_BASE_URL}/{suburb}/crime-weather")
    return response.json() if response.status_code == 200 else {}

suburb_dropdown = widgets.Dropdown(
    options=[suburb['name'] for suburb in get_suburbs()],
    description='Suburb:',
)

# Modify handle_click to use the selected suburb
def handle_click(plot_output, chart_type, data_type=None):
    suburb = suburb_dropdown.value
    with plot_output:
        plot_output.clear_output(wait=True)
        
        if chart_type == 'map_scatter':
            crime_data = get_crime_data(suburb)
            weather_data = get_weather_data(suburb)
            plot_map(data_type, crime_data)
            plot_scatter_chart(data_type, weather_data)
        elif chart_type == 'bar':
            crime_weather_data = get_crime_weather_data(suburb)
            plot_bar_chart(data_type, crime_weather_data)
        elif chart_type == 'pie':
            crime_weather_data = get_crime_weather_data(suburb)
            plot_pie_chart(crime_weather_data)

# Static guide and queries
guide = """
    <b>Welcome to the Victoria Crime Analysis System!</b>
    <br>
    In light of the tragic knife attack at Westfield Bondi Junction in Sydney, there's been a renewed focus across Australia on developing effective crime prevention strategies.
    Both citizens and authorities are actively seeking effective solutions to prevent such alarming events. 
    The <b>Victoria Crime Analysis System</b> is your one-stop shop for data-powered 
    insights into crime trends within Victoria. We leverage big data analytics to uncover factors influencing criminal activity in urban areas.
    <hr>
    🔍 Simply click the search button next to your query, and our system will deliver a real-time response! 
    <hr>
"""
queries = [
    ("<b>Query 1: Do weather patterns influence number of criminal incidents?</b>", [
        ('1.1. Does temperature influence the number of criminal incidents?', 'map_scatter', 'temp'),
        ('1.2. Does precipitation influence the number of criminal incidents?', 'map_scatter', 'precip')
    ]),
    ('<b>Query 2: Do weather patterns influence types of criminal incidents?</b>', [
        ('2.1. Does temperature affect the number of Type A and Type D crimes?', 'bar', 'temp'),
        ('2.2. Does precipitation affect the number of Type A and Type D crimes?', 'bar', 'precip')
    ]),
    ('<b>Query 3: Is public sentiment correlated to number of criminal incidents?</b>', [
        ('3.1. Is public sentiment correlated to number of criminal incidents?', 'pie', None)
    ])
]

# Widgets for display
plot_output = widgets.Output()
guide_html = widgets.HTML(guide)

# Create the rows for each query and sub-query
rows = [guide_html]

for query, sub_queries in queries:
    query_html = widgets.HTML(value=query)
    rows.append(query_html)
    for sub_query, chart_type, data_type in sub_queries:
        sub_query_html = widgets.HTML(value=sub_query)
        button = widgets.Button(description="Search")
        button.on_click(lambda _, ct=chart_type, dt=data_type: handle_click(plot_output, ct, dt))
        rows.append(widgets.HBox(children=[sub_query_html, button]))

# Display Container
container = widgets.VBox(children=[suburb_dropdown] + rows + [plot_output])
display(container)

ConnectionError: HTTPSConnectionPool(host='localhost', port=9090): Max retries exceeded with url: /api/suburbs (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x128b00e80>: Failed to establish a new connection: [Errno 61] Connection refused'))